In [1]:
import numpy as np
import keras as K
import matplotlib.pyplot as plt

%matplotlib inline

Using TensorFlow backend.


In [2]:
%%time
from keras.datasets.cifar10 import load_data
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = load_data()

y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

CPU times: user 84 ms, sys: 44 ms, total: 128 ms
Wall time: 125 ms


In [3]:
def preprocess_img(img):
    img = img.astype(np.float32) / 255.0
    img -= 0.5
    return img * 2

In [49]:
from multiprocessing import Pool

pool = Pool(processes=8)

In [48]:
pool.terminate()

In [57]:
%%time
#from keras.preprocessing.image import ImageDataGenerator

import tools.image as T

# Useful for checking the output of the generators after code change
#from importlib import reload
reload(T)

gen = T.ImageDataGenerator(
     featurewise_center=False,
     samplewise_center=False,
     featurewise_std_normalization=False,
     samplewise_std_normalization=False,
     zca_whitening=False,
     rotation_range=45,
     width_shift_range=.1,
     height_shift_range=.1,
     shear_range=0.,
     zoom_range=0,
     channel_shift_range=0,
     fill_mode='nearest',
     cval=0.,
     horizontal_flip=True,
     vertical_flip=False,
     rescale=None,
#      preprocessing_function=preprocess_img,
     dim_ordering='default',
     pool=pool
)
test_gen = T.ImageDataGenerator(
    preprocessing_function=preprocess_img)

gen.fit(X_train)
X_train_aug = gen.flow(X_train, y_train_cat)
X_test_aug = test_gen.flow(X_test, y_test_cat)

# fig, ax = plt.subplots(6, 6, figsize=(15, 15))
# for (imgs, labels) in X_train_aug:
#     for i in range(6):
#         for j in range(6):
#             if i*6 + j < 32:
#                 ax[i][j].imshow(imgs[i*6 + j])
#     break

for i, (imgs, labels) in enumerate(X_train_aug):
    if i == 1000:
        break

CPU times: user 6.52 s, sys: 8 ms, total: 6.52 s
Wall time: 6.51 s


In [27]:
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, BatchNormalization

# # Create the model
# model = Sequential()
# model.add(Convolution2D(32, 3, 3, input_shape=(3, 32, 32), border_mode='same', activation='relu', W_constraint=maxnorm(3)))
# model.add(Dropout(0.2))
# model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same', W_constraint=maxnorm(3)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Flatten())
# model.add(Dense(512, activation='relu', W_constraint=maxnorm(3)))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))
# # Compile model
# epochs = 25
# lrate = 0.01
# decay = lrate/epochs
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
# model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
# print(model.summary())

model = Sequential([
    Conv2D(32, 3, 3, border_mode='same', activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D(),
    Conv2D(16, 3, 3, border_mode='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, 3, border_mode='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, 3, border_mode='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(10, activation='softmax')
])

model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(X_train_aug, samples_per_epoch=50000, nb_epoch=50, 
                    validation_data=X_test_aug, nb_val_samples=10000)
#model.fit(X_train, y_train_cat, batch_size=512, validation_data=(X_test, y_test_cat), nb_epoch=200)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_37 (Convolution2D) (None, 32, 32, 32)    896         convolution2d_input_10[0][0]     
____________________________________________________________________________________________________
maxpooling2d_37 (MaxPooling2D)   (None, 16, 16, 32)    0           convolution2d_37[0][0]           
____________________________________________________________________________________________________
convolution2d_38 (Convolution2D) (None, 16, 16, 16)    4624        maxpooling2d_37[0][0]            
____________________________________________________________________________________________________
maxpooling2d_38 (MaxPooling2D)   (None, 8, 8, 16)      0           convolution2d_38[0][0]           
___________________________________________________________________________________________

KeyboardInterrupt: 